1. 스팸 메일 데이터 이해

In [ ]:
# !apt-get install -y curl
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
from collections import Counter
from konlpy.tag import Mecab


In [ ]:

# from google.colab import files
# uploaded = files.upload()
# import json

# file_path = '/content/K2-00001-CL33762-CP33206-05-08-S2.json'

# try:
#     with open(file_path, 'r', encoding='utf-8') as file:
#         datas = json.load(file)

#         # 각 sessionInfo에 대해 반복
#         utterance_list = []
#         for session_info in datas.get('sessionInfo', []):
#             dialog_list = session_info.get('dialog', [])

#             # 각 dialog에 대해 반복
#             for dialog in dialog_list:
#                 utterance = dialog.get('utterance', '')
#                 utterance_list.append(utterance)

#         # 각 값을 엔터로 구분하여 출력
#         print('\n'.join(utterance_list))

# except FileNotFoundError:
#     print(f"File not found: {file_path}")
# except Exception as e:
#     print(f"An error occurred: {e}")


In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/tonykorea99/Spam-alart/main/Spam%20Data.csv", filename='Spam%20Data.csv')

data = pd.read_csv('Spam%20Data.csv', encoding='utf-8')

data = data.sample(frac=1, random_state=42)

print('총 샘플의 수 :', len(data))

In [ ]:
data[:5]

In [ ]:
# 레이블과 메일 내용이 담긴 v1, v2 열만 필요 : unnamed 2-4 열은 삭제
# 레이블 값을 0(ham), 1(spam)으로 변경


data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data[1199:1204]

In [ ]:
# data의 정보 확인

data.info()

In [ ]:
# null 값 포함 여부 확인

data.isnull().values.any()

In [ ]:
# 중복 데이터 확인

data['v2'].nunique(), data['v1'].nunique()

In [ ]:
# 중복 데이터 제거

data.drop_duplicates(subset=['v2'], inplace=True)

print('총 샘플의 수 :',len(data))

In [ ]:
# 레이블 값의 분포 시각화

data['v1'].value_counts().plot(kind='bar')

In [ ]:
# 스팸 메일과 정상 메일 수치로 확인

print(data.groupby('v1').size().reset_index(name='count'))

In [ ]:
# X와 y를 분리합니다. v2열을 X, v1열을 y로 저장

X_data = data['v2']
y_data = data['v1']

print('메일 본문의 개수: {}'.format(len(X_data)))
print('레이블의 개수: {}'.format(len(y_data)))

In [ ]:
# 토큰화, 정수 인코딩

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data) # 행을 가진 X의 각 행에 토큰화를 수행
sequences = tokenizer.texts_to_sequences(X_data) # 단어를 숫자값, 인덱스로 변환하여 저장

In [ ]:
print(sequences[:5])

In [ ]:
# 각 정수가 어떤 단어에 부여되었는지 확인

word_to_index = tokenizer.word_index
print(word_to_index)

In [ ]:
# tokenizer.word_counts.items()를 사용하여 각 단어의 등장 빈도수 확인

threshold = 2
total_cnt = len(word_to_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
#단어 집합의 크기를 제한하진 않고 답어 집합 크기 설정

vocab_size = len(word_to_index) + 1   # 패딩을 위한 토큰인 0번 단어를 고려하며 +1을 해서 저장
print('단어 집합의 크기: {}'.format((vocab_size)))

In [ ]:
# 훈련 데이터와 테스트 데이터 8:2 분리

n_of_train = int(len(sequences) * 0.8)
n_of_test = int(len(sequences) - n_of_train)
print('훈련 데이터의 개수 :',n_of_train)
print('테스트 데이터의 개수:',n_of_test)

In [ ]:
# X_data에 대해서 정수 인코딩 된 결과인 sequences를 X_data로 변경하고,
# 전체 데이터에서 가장 길이가 긴 메일과 전체 메일 데이터의 길이 분포 확인

X_data = sequences
print('메일의 최대 길이 : %d' % max(len(l) for l in X_data))
print('메일의 평균 길이 : %f' % (sum(map(len, X_data))/len(X_data)))
plt.hist([len(s) for s in X_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show();

In [ ]:
# 패딩

max_len = 559
# 전체 데이터셋의 길이는 max_len으로 맞춥니다.
data = pad_sequences(X_data, maxlen = max_len)
print("훈련 데이터의 크기(shape): ", data.shape)

In [ ]:
# 훈련 데이터(X_train)와 테스트 데이터(X_test) 분리

X_test = data[n_of_train:]
y_test = np.array(y_data[n_of_train:])
X_train = data[:n_of_train]
y_train = np.array(y_data[:n_of_train])

2. biLSTM으로 스팸 메일 분류하기

In [ ]:
# 모델 설계

In [ ]:
import re
from tensorflow.keras.layers import Embedding, Dense,LSTM , Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import f1_score as sklearn_f1_score
from keras.metrics import Precision, Recall

In [ ]:
from keras import backend as K

def custom_f1_score(y_true, y_pred):
    y_true = K.cast(K.round(y_true), 'float32')
    y_pred = K.cast(K.round(y_pred), 'float32')

    tp = K.sum(y_true * y_pred, axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)

In [ ]:
embedding_dim = 100
hidden_units =128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Bidirectional(LSTM(hidden_units)))  # BiLSTM사용
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

In [ ]:
# 모델 훈련

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc', Precision(), Recall(), custom_f1_score])
# history = model.fit(X_train, y_train, epochs=4, batch_size=64, validation_split=0.2)





In [ ]:
# 테스트 데이터에 대한 정확도

result = model.evaluate(X_test, y_test, return_dict=True)
print("\n 테스트 결과:", result)

In [ ]:
# 훈련 데이터와 검증 데이터에 대한 정확도 비교

epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show();

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))